In [1]:
import os
import mat_properties as prop
import numpy as n
import pandas as pd
import cotel
import nasos
import time
from scipy.optimize import root

#Время начала расчёта
start_time = time.time()

#Основные константы
calctolerance=10**-3
calcmethod = "hybr"
KPDnasos = 0.9
KPD=0.99

#таблица ном режима
gas_streams0   = pd.read_excel('streams0.xlsx', sheet_name='gas',  index_col=0)
water_streams0 = pd.read_excel('streams0.xlsx', sheet_name='water',index_col=0)
#таблица задаваемых значений (заменится на интерфейс)
gas_streamsZAD    = pd.read_excel('streams.xlsx',  sheet_name='gas',  index_col=0)
water_streamsZAD  = pd.read_excel('streams.xlsx',  sheet_name='water',index_col=0)
#рабочая таблица (=номинал в 1 итерации)
gas_streams = gas_streams0.copy()
water_streams = water_streams0.copy()


#чтение с вводного экселя входных данных
Tgas = gas_streamsZAD.loc['GTU-PEVD','T']
Pgas = gas_streamsZAD.loc['GTU-PEVD','P']
Ggas = gas_streamsZAD.loc['GTU-PEVD','G']
Propgas = gas_streamsZAD.loc['GTU-PEVD','N2':'Ar']
Pvd = water_streamsZAD.loc['IVD-PEVD','P']
Pnd = water_streamsZAD.loc['IND-PPND','P']
Twat0 = water_streamsZAD.loc['X-GPK','T']

#запись состава газов в раб табл
gas_streams['N2'] = Propgas[0]
gas_streams['O2'] = Propgas[1]
gas_streams['CO2'] = Propgas[2]
gas_streams['H2O'] = Propgas[3]
gas_streams['Ar'] = Propgas[4]


#Состав газов в номинальном режиме и в частичной нагрузке
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
fractiongas0 = (gas_streams0.at['GTU-PEVD','N2'],gas_streams0.at['GTU-PEVD','O2'],gas_streams0.at['GTU-PEVD','CO2'],gas_streams0.at['GTU-PEVD','H2O'],gas_streams0.at['GTU-PEVD','Ar'])
fractiongas  = (gas_streams.at['GTU-PEVD','N2'],gas_streams.at['GTU-PEVD','O2'],gas_streams.at['GTU-PEVD','CO2'],gas_streams.at['GTU-PEVD','H2O'],gas_streams.at['GTU-PEVD','Ar'])

#Считывание рефпропа
RP = prop.init_REFPROP(r'C:\Program Files (x86)\REFPROP')
os.environ['RPPREFIX'] = r'C:/Program Files (x86)/REFPROP'
gas1 = prop.Materials_prop(gasmix,fractiongas, prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)
gas0 = prop.Materials_prop(gasmix,fractiongas0, prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)
water=prop.Materials_prop('water',[1.0,0,0,0,0], prop.REFPROP_h_s,prop.REFPROP_p_t,prop.REFPROP_p_h,prop.REFPROP_p_s,prop.REFPROP_p_q,prop.REFPROP_t_q,RP=RP)

#Задание в раб табл параметров газа (1 строка)
gas_streams.loc['GTU-PEVD','T'] = Tgas
gas_streams['G'] = Ggas
gas_streams['P'] = Pgas
gas_streams.loc['GTU-PEVD','H'] = gas1.p_t(Pgas,Tgas)['h']

#Задание в раб табл параметров по воде
water_streams.loc['IVD-PEVD':'PEN-EVD','P'] = Pvd
water_streams.loc['IND-PPND':'X-GPK','P'] = Pnd
water_streams.loc['BND-PEN','P'] = Pnd
water_streams.loc['X-GPK','T'] = Twat0

#Расчёт параметров на насыщении в испарительных поверхностях (недогрев до нас на входе 3°С)
water_streams.loc['IVD-PEVD','T'] = water.p_q(Pvd,1)['T']
water_streams.loc['IVD-PEVD','H'] = water.p_q(Pvd,1)['h']
water_streams.loc['IND-PPND','T'] = water.p_q(Pnd,1)['T']
water_streams.loc['IND-PPND','H'] = water.p_q(Pnd,1)['h']
water_streams.loc['EVD-IVD','T'] = water.p_q(Pvd,1)['T']-3
water_streams.loc['GPK-IND','T'] = water.p_q(Pnd,1)['T']-3
water_streams.loc['EVD-IVD','H'] = water.p_t(Pvd,water_streams.loc['EVD-IVD','T'])['h']
water_streams.loc['GPK-IND','H'] = water.p_t(Pnd,water_streams.loc['GPK-IND','T'])['h']


it=30




######################################################################
#Связка низкого давления
for j in range(it):
    #Связка ППНД+ИНД
    for i in range(it):
        #Расчёт ППНД
        PPND = cotel.heatex('EVD-PPND','PPND-IND','IND-PPND','PPND-TURB',    KPD,calctolerance,gas0,gas1,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
        gas_streams.loc['PPND-IND','T':'G']    = [PPND[0],PPND[1],PPND[2],PPND[3]]
        water_streams.loc['PPND-TURB','T':'G'] = [PPND[4],PPND[5],PPND[6],PPND[7]]
        #Расчёт ИНД
        IND  = cotel.vaporND('PPND-IND','IND-GPK','GPK-IND','IND-PPND',      KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
        gas_streams.loc['IND-GPK','T':'G']    = [IND[0],IND[1],IND[2],IND[3]]
        water_streams.loc['IND-PPND','T':'G'] = [IND[4],IND[5],IND[6],IND[7]]
        #Переопределение расхода пара
        water_streams.loc['PPND-TURB':'IND-PPND','G'] = IND[7]
        Qgas = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['IND-GPK','H'])
        Qwat = water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
        print('dQ/Q ППНД+ИНД',((Qgas-Qwat)/Qgas))
        if  abs((Qgas-Qwat)/Qgas) < calctolerance:
            break;
    #ПЭН
    water_streams.loc['BND-PEN','T':'G'] = [IND[9],IND[10],IND[11],IND[12]]
    PEN=nasos.nasos('BND-PEN','PEN-EVD',water,KPDnasos,water_streams).calc()
    water_streams.loc['PEN-EVD','T':'G'] = [PEN[0],PEN[1],PEN[2],PEN[3]]

    #Расчёт расхода в ГПК
    water_streams.loc['X-GPK','G'] = water_streams.at['PPND-TURB','G']+water_streams.at['PEVD-DROS','G']
    water_streams.loc['GPK-IND','G'] = water_streams.at['X-GPK','G']
    #Расчёт ГПК
    GPK = cotel.heatex('IND-GPK','GPK-out','X-GPK','GPK-IND',           KPD,calctolerance,gas1,gas0,water,calcmethod,gas_streams0,water_streams0,gas_streams,water_streams).calc()
    gas_streams.loc['GPK-out','T':'G']   = [GPK[0],GPK[1],GPK[2],GPK[3]]
    water_streams.loc['GPK-IND','T':'G'] = [GPK[4],GPK[5],GPK[6],GPK[7]]

    Qgas1 = KPD*gas_streams.at['EVD-PPND','G']*(gas_streams.at['EVD-PPND','H']-gas_streams.at['GPK-out','H'])
    Qwat1 = water_streams.at['GPK-IND','G']*(water_streams.at['GPK-IND','H']-water_streams.at['X-GPK','H'])+water_streams.at['IND-PPND','G']*(water_streams.at['IND-PPND','H']-water_streams.at['GPK-IND','H'])+water_streams.at['PPND-TURB','G']*(water_streams.at['PPND-TURB','H']-water_streams.at['IND-PPND','H'])+water_streams.at['BND-PEN','G']*(water_streams.at['BND-PEN','H']-water_streams.at['GPK-IND','H'])
    print('dQ/Q ППНД+ИНД+ГПК',(Qgas1-Qwat1)/Qgas1)
    if  abs((Qgas1-Qwat1)/Qgas1) < calctolerance:
        break;
print("НД+ --- %s сек. ---" % round((time.time() - start_time)))
######################################################################


dQ/Q ППНД+ИНД -0.0016489318859761685
dQ/Q ППНД+ИНД -0.001900519303481959
dQ/Q ППНД+ИНД -0.0018955414099058624
dQ/Q ППНД+ИНД -0.0018956401307442397
dQ/Q ППНД+ИНД -0.0018956381728634747
dQ/Q ППНД+ИНД -0.0018956382120209104
dQ/Q ППНД+ИНД -0.0018956382107457891
dQ/Q ППНД+ИНД -0.0018956382110170387
dQ/Q ППНД+ИНД -0.0018956382108974669
dQ/Q ППНД+ИНД -0.0018956382109218022


KeyboardInterrupt: 